In [1]:
from tensorflow.keras.models import load_model

In [2]:
import cv2
import numpy as np

In [3]:
model = load_model('mask_detector_cnn.h5')

In [4]:
face = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [5]:
source = cv2.VideoCapture(0)

In [6]:
labels_dict = {0 :'WithMask',1:'WearAMask!!'}
color_dict = {0:(0,255,0),1:(0,0,255)}

In [7]:
while(True):
    ret,img = source.read()
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face.detectMultiScale(gray,1.3,5)
    
    for x,y,w,h in faces:
        face_img = gray[y:y+w,x:x+w]
        resized = cv2.resize(face_img,(112,112))
        normalized = resized/255.0
        reshaped = np.reshape(normalized,(1,112,112,1))
        result = model.predict(reshaped)
        
        label=np.argmax(result,axis=1)[0]
        
        cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(img,labels_dict[label],(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(0,0,0),2)
        
    cv2.imshow('LIVE',img)
    key=cv2.waitKey(1)
    
    
    if(key==27):
        break
        
        
cv2.destroyAllWindows()
source.release()